In [1]:
import numpy as np
import gzip
# Using file output from FLASH (See Amplicon sequencing and data analysis)
f = gzip.open('./P3.extendedFrags.fastq.gz') # Example from eLIDAR cells cultured with ligand
#f = gzip.open('./N3.extendedFrags.fastq.gz') # Example from eLIDAR cells cultured without ligand
doc1 = f.readlines()
f.close()

In [2]:
total_reads = len(doc1)/4
print('total # of reads:',total_reads)

total # of reads: 334022.0


In [3]:
#  function to convert chars to score
def string2ascii(s, l):
    scores = [ord(c)-33 for c in s]
    # if average score of window (3) is < 27 = low confidence, else return average score of all read
    for i in range(len(scores)-3):
        if ((scores[i]+scores[i+1]+scores[i+2]+scores[i+3])/3)<27:
            return 1
    return sum(scores)/l 

In [4]:
# filter reads by score
seq_only=[]
seq_l=[]
smaller=0
for i in range(0, len(doc1[:]), 4):
    seq1 = doc1[i+1]
    seq1 = str(seq1[:-1], encoding='utf-8')
    score1 = doc1[i+3]
    score1 = str(score1[:-1], encoding='utf-8')
    l = len(seq1)
    if l < 272: #272 is the minimum size we accept (to capture all of the stemloop that contain stop codon, etc)
        smaller+=1
        continue
    if string2ascii(score1,l) > 30: # 1 in 1000
        seq_only.append(seq1)
        seq_l.append(l)
print('High quality reads:',len(seq_l))
print("--> Fraction of high quality reads:", len(seq_l)/total_reads)
print("Reads not long enough:", smaller)

High quality reads: 304176
--> Fraction of high quality reads: 0.9106466041158966
Reads not long enough: 7114


In [5]:
np.median(seq_l)

356.0

In [6]:
np.std(seq_l)

6.869340967595127

In [7]:
c = 0
ed = 0
non = 0
non_seq = []

#Only looking at forward strand as FLASH merged the pair reads, leaving us with only forward strands. 
#Given that there are no other "A", we should not see any edit and if we do we wouldn't be able to tell if
#it was caused by ADAR so we discard it!
for s1 in seq_only[:]:
  if   'CCTCCGTTTAGGTGGGTGG' in str(s1):
    c=c+1
  elif 'CCTCCGTTTGGGTGGGTGG' in str(s1):
    ed=ed+1
  else:
    non=non+1
    non_seq.append(s1)

print('Not edited:',c)
print('Edited:',ed)
print('Non-amplicon reads:',non) # reads that didn't map to sequence (~1-2%)
print('Total reads:',c+ed+non, end='\n\n')

print('% of non-amplicon reads:', 100*non/(c+ed+non))
print('% of edited reporter (all reads)', 100*ed/(c+ed+non))
print('% of not edited reporter (all reads)', 100*c/(c+ed+non))

#we care about the % of reads that are edit and were mapped to sequence
print('----> % of edited reporter (amplicon reads)', 100*ed/(c+ed)) 
print('% of not edited reporter (amplicon reads)', 100*c/(c+ed))

Not edited: 298146
Edited: 698
Non-amplicon reads: 5332
Total reads: 304176

% of non-amplicon reads: 1.7529325127557729
% of edited reporter (all reads) 0.22947241070958918
% of not edited reporter (all reads) 98.01759507653463
----> % of edited reporter (amplicon reads) 0.2335666769284309
% of not edited reporter (amplicon reads) 99.76643332307157


In [51]:
#eLIDAR
#N1 = 3.3681
#n2 = 3.5284
#n3 = 3.3679
#p1 = 0.2618
#p2 = 0.2745
#p3 = 0.2335